In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns

import sys
sys.path.append('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/src/')
import _config

sys.path.append('/home/unix/maxwshen/')
from mylib import util

Using data folder:
 /ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/data/


In [2]:
## Load design df

modelexp_df = pd.read_csv(_config.DATA_DIR + f'modelexp_intsimple_rs.csv')
datagroup_df = pd.read_csv(_config.DATA_DIR + f'datagroup_intsimple.csv')

modelexp_df = modelexp_df.merge(datagroup_df, on = 'dataset')

modelexp_df.head()

,random_seed,dataset,Name,datasource_x,Unnamed: 0,data_readlen,data_noise,data_num_proposed_gts,data_proposal_type,data_num_groups,data_risingstar_num,datasource_y
0,0,intsimple_smart_rl_1_noise_0_star_3,0,pb_interpolate,0,1,0.0,0,smart,19,3,pb_interpolate
1,0,intsimple_smart_rl_1_noise_0_star_4,1,pb_interpolate,1,1,0.0,0,smart,19,4,pb_interpolate
2,0,intsimple_smart_rl_1_noise_0_star_5,2,pb_interpolate,2,1,0.0,0,smart,19,5,pb_interpolate
3,0,intsimple_smart_rl_1_noise_0_star_6,3,pb_interpolate,3,1,0.0,0,smart,19,6,pb_interpolate
4,0,intsimple_smart_rl_1_noise_0_star_7,4,pb_interpolate,4,1,0.0,0,smart,19,7,pb_interpolate


In [3]:
## Load ground truth data

fitness_df = pd.read_csv('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_pt/fullgt_fitness.csv', index_col = 0)
fitness_df.head()

fq_df = pd.read_csv('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/pb_e3_interpolate_multi/badran_pacbio_pivot_1pct.csv')
time_cols = [col for col in fq_df.columns if 'hrs' in col]
fq_df = fq_df.rename(columns = {col: str(idx) for idx, col in enumerate(time_cols)})
fq_df = fq_df.set_index('Abbrev genotype')

In [4]:
fq_df.head()

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
Abbrev genotype,,,,,,,,,,,,,,,,,,,,,
VIWS.DNGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.010436,0.101351,0.307207,0.516204,0.690665,0.806228,0.840179,0.858280,0.859680,0.800000
VIW..DNGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007699,0.067568,0.100481,0.082836,0.054376,0.031142,0.035129,0.038845,0.042116,0.042424
VIW...N.ERI.YC.KSKL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.211217,0.067568,0.007564,0.000469,0.000023,0.000000,0.000014,0.000186,0.002463,0.030303
VIWS..NGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012949,0.135135,0.240628,0.237527,0.186697,0.128028,0.088182,0.059538,0.039415,0.024242
VIWS.DNGE.I.YC.KSKL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000012,0.000144,0.001679,0.018182


In [5]:
int_tp = 16

cols = [str(s) for s in range(int_tp + 1)]
keep_gts = list(fq_df[fq_df[cols].apply(sum, axis = 'columns') > 0.01].index)
keep_gts

['...................',
 'V...........YC.....',
 'V............C.....',
 '............YC.....',
 'V..................',
 'V...........Y......',
 '............Y......',
 'VI.................']

## Evaluate single prediction

In [6]:
row = modelexp_df[modelexp_df['Name'] == 79].iloc[0]
print(row)

model_fold = '/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_from_reads_pt_multi/modelexp_intsimple_rs/'
model_nm = row['Name']

pred_fitness_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_fitness.csv', index_col = 0)
pred_fq_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_genotype_matrix.csv', index_col = 0)

last_t_col = pred_fq_df.columns[-1]
pfqdfs = pred_fq_df.sort_values(by = last_t_col, ascending = False)
consensus_gt = pfqdfs.iloc[0].name
consensus_fq = pfqdfs[last_t_col].iloc[0]

pred_consensus_fitness = pred_fitness_df[pred_fitness_df['Genotype'] == consensus_gt]['Inferred fitness'].iloc[0]

# Predict consensus gt as rising star if fq < 0.50
second_last_t_col = pred_fq_df.columns[-2]
is_rising = pred_fq_df.loc[consensus_gt, last_t_col] - pred_fq_df.loc[consensus_gt, second_last_t_col]
not_consensus = bool(pred_fq_df.loc[consensus_gt, last_t_col] < 0.50)
if is_rising and not_consensus:
    pred_fitness_df['Rising star, predicted'] = (pred_fitness_df['Inferred fitness'] >= pred_consensus_fitness)
else:
    pred_fitness_df['Rising star, predicted'] = (pred_fitness_df['Inferred fitness'] > pred_consensus_fitness)

# Get observed rising stars
last_t_col = pred_fq_df.columns[-1]
second_last_t_col = pred_fq_df.columns[-2]
print()

ofqdfs = fq_df.sort_values(by = last_t_col, ascending = False)
obs_consensus_gt = ofqdfs.iloc[0].name
obs_is_rising = fq_df.loc[obs_consensus_gt, last_t_col] - fq_df.loc[obs_consensus_gt, second_last_t_col]
obsnot_consensus = bool(fq_df.loc[obs_consensus_gt, last_t_col] < 0.50)

obs_consensus_fitness = fitness_df[fitness_df['Genotype'] == obs_consensus_gt]['Fitness'].iloc[0]
int_tp = row['data_risingstar_num']

# Rising in last two timepoints
crit = (fq_df[str(int_tp)] - fq_df[str(int_tp - 1)] > 0)
keep_gts_crit3 = set(fq_df[crit].index)

# Above 3% frequency in last timepoint
last_timepoint_min_fq = 0.03
keep_gts_crit4 = set(fq_df[fq_df[str(int_tp)] >= last_timepoint_min_fq].index)

keep_gts = keep_gts_crit3 & keep_gts_crit4
crit = (fitness_df['Genotype'].isin(keep_gts))
dfs = fitness_df[crit]

# Label most frequent gt as rising star if fq < 0.50 and fq > 0.50 later
if obs_is_rising and obsnot_consensus:
    dfs['Rising star, observed'] = (dfs['Fitness'] >= obs_consensus_fitness)
else:
    dfs['Rising star, observed'] = (dfs['Fitness'] > obs_consensus_fitness)
        
        
pred_df = pred_fitness_df.merge(dfs, on = 'Genotype', how = 'outer')
pred_df['Rising star, observed'] = pred_df['Rising star, observed'].fillna(value = False)
pred_df['Rising star, predicted'] = pred_df['Rising star, predicted'].fillna(value = False)
display(pred_df)

random_seed                                                 0
dataset                  intsimple_smart_rl_1_noise_0_star_82
Name                                                       79
datasource_x                                   pb_interpolate
Unnamed: 0                                                 79
data_readlen                                                1
data_noise                                                  0
data_num_proposed_gts                                      80
data_proposal_type                                      smart
data_num_groups                                            19
data_risingstar_num                                        82
datasource_y                                   pb_interpolate
Name: 79, dtype: object



/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Genotype,Inferred fitness,"Rising star, predicted",Fitness,"Rising star, observed"
0,......N.ERI.YC.KSKL,1.193864,False,NaN,False
1,.I.................,0.801435,False,NaN,False
2,V.W.....E...YC.KS..,0.931762,False,NaN,False
3,VI.........PYCDK...,0.835063,False,NaN,False
4,VIW.G.NGE.IPYCDKS.L,0.934782,False,NaN,False
5,VI....N...I.YC.KS.L,1.140877,False,NaN,False
6,.......G...........,0.854843,False,NaN,False
7,V.W.......I.YC.K...,1.072501,False,NaN,False
8,VI..........YC.K...,0.920573,False,NaN,False
9,V...........YC.KS..,0.903711,False,NaN,False


In [7]:
# AUROC, AUPRC

pred_df['Inferred fitness'] = pred_df['Inferred fitness'].fillna(value = 0)

from sklearn.metrics import roc_auc_score
auroc = roc_auc_score(
    pred_df['Rising star, observed'],
    pred_df['Inferred fitness'],
)
print(auroc)

from sklearn.metrics import average_precision_score
aps = average_precision_score(
    pred_df['Rising star, observed'],
    pred_df['Inferred fitness'],
)
print(aps)

1.0
1.0


## evaluate many predictions

In [18]:
model_fold = '/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_from_reads_pt_multi/modelexp_intsimple_rs/'

from collections import defaultdict
stats_dd = defaultdict(list)
crit = (modelexp_df['data_readlen'] == 1) & (modelexp_df['data_noise'] == 0.05)
for idx, row in modelexp_df[crit].iterrows():
    model_nm = row['Name']

    pred_fitness_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_fitness.csv', index_col = 0)
    pred_fq_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_genotype_matrix.csv', index_col = 0)
    if len(pred_fq_df) == 0:
        continue
    
    last_t_col = pred_fq_df.columns[-1]
    pfqdfs = pred_fq_df.sort_values(by = last_t_col, ascending = False)
    consensus_gt = pfqdfs.iloc[0].name
    consensus_fq = pfqdfs[last_t_col].iloc[0]

    pred_consensus_fitness = pred_fitness_df[pred_fitness_df['Genotype'] == consensus_gt]['Inferred fitness'].iloc[0]

    # Predict consensus gt as rising star if fq < 0.50
    second_last_t_col = pred_fq_df.columns[-2]
    is_rising = pred_fq_df.loc[consensus_gt, last_t_col] - pred_fq_df.loc[consensus_gt, second_last_t_col]
    not_consensus = bool(pred_fq_df.loc[consensus_gt, last_t_col] < 0.50)
    if is_rising and not_consensus:
        pred_fitness_df['Rising star, predicted'] = (pred_fitness_df['Inferred fitness'] >= pred_consensus_fitness)
    else:
        pred_fitness_df['Rising star, predicted'] = (pred_fitness_df['Inferred fitness'] > pred_consensus_fitness)

        
    # Get observed rising stars
    last_t_col = pred_fq_df.columns[-1]
    second_last_t_col = pred_fq_df.columns[-2]

    ofqdfs = fq_df.sort_values(by = last_t_col, ascending = False)
    obs_consensus_gt = ofqdfs.iloc[0].name
    obs_is_rising = fq_df.loc[obs_consensus_gt, last_t_col] - fq_df.loc[obs_consensus_gt, second_last_t_col]
    obsnot_consensus = bool(fq_df.loc[obs_consensus_gt, last_t_col] < 0.50)

    obs_consensus_fitness = fitness_df[fitness_df['Genotype'] == obs_consensus_gt]['Fitness'].iloc[0]
    int_tp = row['data_risingstar_num']

    # Rising in last two timepoints
    crit = (fq_df[str(int_tp)] - fq_df[str(int_tp - 1)] > 0)
    keep_gts_crit3 = set(fq_df[crit].index)

    # Above 3% frequency in last timepoint
    last_timepoint_min_fq = 0.03
    keep_gts_crit4 = set(fq_df[fq_df[str(int_tp)] >= last_timepoint_min_fq].index)

    keep_gts = keep_gts_crit3 & keep_gts_crit4
    crit = (fitness_df['Genotype'].isin(keep_gts))
    dfs = fitness_df[crit]

    # Label most frequent gt as rising star if fq < 0.50 and fq > 0.50 later
    if obs_is_rising and obsnot_consensus:
        dfs['Rising star, observed'] = (dfs['Fitness'] >= obs_consensus_fitness)
    else:
        dfs['Rising star, observed'] = (dfs['Fitness'] > obs_consensus_fitness)

        
    # Combine
    pred_df = pred_fitness_df.merge(dfs, on = 'Genotype', how = 'outer')
    pred_df['Rising star, observed'] = pred_df['Rising star, observed'].fillna(value = False)
    pred_df['Rising star, predicted'] = pred_df['Rising star, predicted'].fillna(value = False)
    
    
    # Record stats
    stats_dd['random_seed'].append(row['random_seed'])
    stats_dd['data_risingstar_num'].append(row['data_risingstar_num'])
    stats_dd['Num pred rising stars'].append(sum(pred_df['Rising star, predicted']))
    stats_dd['Num true rising stars'].append(sum(pred_df['Rising star, observed']))
    
    crit = (pred_df['Rising star, predicted'] == True) & (pred_df['Rising star, observed'] == True)
    stats_dd['True positive'].append(sum(crit))

    crit = (pred_df['Rising star, predicted'] == True) & (pred_df['Rising star, observed'] == False)
    stats_dd['False positive'].append(sum(crit))
    
    crit = (pred_df['Rising star, predicted'] == False) & (pred_df['Rising star, observed'] == False)
    stats_dd['True negative'].append(sum(crit))
    
    crit = (pred_df['Rising star, predicted'] == False) & (pred_df['Rising star, observed'] == True)
    stats_dd['False negative'].append(sum(crit))
    
    # AUROC, AUPRC
    pred_df['Inferred fitness'] = pred_df['Inferred fitness'].fillna(value = 0)

    from sklearn.metrics import roc_auc_score
    try:
        auroc = roc_auc_score(
            pred_df['Rising star, observed'],
            pred_df['Inferred fitness'],
        )
    except ValueError:
        auroc = np.nan
    stats_dd['AUROC'].append(auroc)

    from sklearn.metrics import average_precision_score
    try:
        aps = average_precision_score(
            pred_df['Rising star, observed'],
            pred_df['Inferred fitness'],
        )
    except ValueError:
        auroc = np.nan
    stats_dd['Average precision score'].append(aps)

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/sklearn/metrics/rank

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [19]:
stats_df = pd.DataFrame(stats_dd)
stats_df['Sensitivity'] = stats_df['True positive'] / (stats_df['True positive'] + stats_df['False negative'])
stats_df['Specificity'] = stats_df['True negative'] / (stats_df['True negative'] + stats_df['False positive'])

stats_df['Precision'] = stats_df['True positive'] / (stats_df['True positive'] + stats_df['False positive'])
stats_df['Recall'] = stats_df['True positive'] / (stats_df['True positive'] + stats_df['False negative'])

stats_df.shape

(75, 14)

In [20]:
dfs = stats_df[stats_df['random_seed'] == 0]
dfs

,random_seed,data_risingstar_num,Num pred rising stars,Num true rising stars,True positive,False positive,True negative,False negative,AUROC,Average precision score,Sensitivity,Specificity,Precision,Recall
0,0,14,1,1,1,0,1,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0,15,1,1,1,0,1,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,0,16,1,1,1,0,1,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,0,17,2,5,1,1,1,4,0.100000,0.671429,0.200000,0.500000,0.500000,0.200000
4,0,18,3,5,2,1,1,3,0.400000,0.828571,0.400000,0.500000,0.666667,0.400000
5,0,19,7,3,3,4,0,0,0.750000,0.638889,1.000000,0.000000,0.428571,1.000000
6,0,20,1,1,1,0,6,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,0,21,0,0,0,0,7,0,NaN,NaN,NaN,1.000000,NaN,NaN
8,0,22,0,0,0,0,7,0,NaN,NaN,NaN,1.000000,NaN,NaN
9,0,23,0,0,0,0,7,0,NaN,NaN,NaN,1.000000,NaN,NaN


In [16]:
dfs[['Precision', 'Recall', 'AUROC', 'Average precision score']].describe()

,Precision,Recall,AUROC,Average precision score
count,41.000000,43.000000,43.000000,43.000000
mean,0.443089,0.341750,0.501653,0.487021
std,0.443095,0.390087,0.380532,0.345519
min,0.000000,0.000000,0.000000,0.023810
25%,0.000000,0.000000,0.123750,0.173423
50%,0.500000,0.200000,0.464286,0.452381
75%,1.000000,0.666667,0.920000,0.791667
max,1.000000,1.000000,1.000000,1.000000


## obs rising stars

In [9]:
model_fold = '/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_from_reads_pt_multi/modelexp_ill_risingstar_rs/'

from collections import defaultdict
for idx, row in modelexp_df[modelexp_df['random_seed'] == 0].iterrows():
    model_nm = row['Name']

    pred_fitness_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_fitness.csv', index_col = 0)
    pred_fq_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_genotype_matrix.csv', index_col = 0)

    last_t_col = pred_fq_df.columns[-1]
    second_last_t_col = pred_fq_df.columns[-2]

    ofqdfs = fq_df.sort_values(by = last_t_col, ascending = False)
    obs_consensus_gt = ofqdfs.iloc[0].name
    obs_is_rising = fq_df.loc[obs_consensus_gt, last_t_col] - fq_df.loc[obs_consensus_gt, second_last_t_col]
    obsnot_consensus = bool(fq_df.loc[obs_consensus_gt, last_t_col] < 0.50)
    
    obs_consensus_fitness = fitness_df[fitness_df['Genotype'] == obs_consensus_gt]['Fitness'].iloc[0]

#     # Cumulative frequency above 1%
    int_tp = row['data_risingstar_num']
#     cols = [str(s) for s in range(int_tp + 1)]
#     keep_gts_crit1 = set(fq_df[fq_df[cols].apply(sum, axis = 'columns') > 0.01].index)

#     # Non-zero frequency in last two timepoints
#     last_two_cols = [str(int_tp - 1), str(int_tp)]
#     keep_gts_crit2 = set(fq_df[fq_df[last_two_cols].astype(bool).apply(sum, axis = 'columns') >= 2].index)

    # Rising in last two timepoints
    crit = (fq_df[str(int_tp)] - fq_df[str(int_tp - 1)] > 0)
    keep_gts_crit3 = set(fq_df[crit].index)
    
    # Above 3% frequency in last timepoint
    last_timepoint_min_fq = 0.03
    keep_gts_crit4 = set(fq_df[fq_df[str(int_tp)] >= last_timepoint_min_fq].index)
             
    keep_gts = keep_gts_crit3 & keep_gts_crit4
#     keep_gts = keep_gts_crit1 & keep_gts_crit2 & keep_gts_crit3 & keep_gts_crit4
    crit = (fitness_df['Genotype'].isin(keep_gts))
    dfs = fitness_df[crit]

    # Label most frequent gt as rising star if fq < 0.50 and fq > 0.50 later
    if obs_is_rising and obsnot_consensus:
        dfs['Rising star, observed'] = (dfs['Fitness'] >= obs_consensus_fitness)
    else:
        dfs['Rising star, observed'] = (dfs['Fitness'] > obs_consensus_fitness)

    print(row['data_risingstar_num'])
    display(dfs[dfs['Rising star, observed']])

5


/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Genotype,Fitness,"Rising star, observed"
87,V..................,0.308412,True


6


/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Genotype,Fitness,"Rising star, observed"


33


/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Genotype,Fitness,"Rising star, observed"


## new

In [ ]:
def get_table_stats(query_gt):
    '''
        - Finds timepoint of peak frequency in ground truth fq matrix
        - Finds earliest timepoint where query_gt was identified as a rising star, or None
        - Calculates advance notice (from above two)
        - Finds ground truth full-length frequency at earliest timepoint
        - Calculates fitness ratio to highest fq. full length genotype at earliest timepoint
    '''